In [ ]:
%cd ..

import src.fetch

get_place_id("borcherdt berlin")

c:\Users\robin\Desktop
--- RESTAURANT DETAILS ---
Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ChIJVfEmINtRqEcRsch0pslaanQ
--- END ---
 


('ChIJVfEmINtRqEcRsch0pslaanQ',
 'Borchardt',
 'Französische Str. 47, 10117 Berlin, Deutschland')

c:\Users\robin\Desktop\Bootcamp_Files
--- RESTAURANT DETAILS ---
Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ChIJVfEmINtRqEcRsch0pslaanQ
--- END ---
 


('ChIJVfEmINtRqEcRsch0pslaanQ',
 'Borchardt',
 'Französische Str. 47, 10117 Berlin, Deutschland')

In [2]:
rest1 = get_place_reviews(get_place_id("funkyfish berlin"))
rest2 = get_place_reviews(get_place_id("standard pizza west berlin"))
rest3 = get_place_reviews(get_place_id("borschert berlin"))

--- RESTAURANT DETAILS ---
FunkyFisch
Kantstraße 135-136, 10625 Berlin, Deutschland
ChIJU7-KvONQqEcRCmPCCvo-Fg4
--- END ---
 
✅ Google reviews retrieved, found 5 reviews for FunkyFisch
 
--- RESTAURANT DETAILS ---
Standard Serious Pizza Charlottenburg
Schlüterstraße 63, 10625 Berlin, Deutschland
ChIJc2uwDI1RqEcRfAsy3YxyfIY
--- END ---
 
✅ Google reviews retrieved, found 5 reviews for Standard Pizza West
 
--- RESTAURANT DETAILS ---
Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ChIJVfEmINtRqEcRsch0pslaanQ
--- END ---
 
✅ Google reviews retrieved, found 5 reviews for Borchardt
 


In [ ]:
#Functions to extract dish mentions from reviews

# services/nlp_service.py
from __future__ import annotations
from typing import Iterable, Dict, List, Set, Tuple
from transformers import pipeline
import re

# ---- Config ----
MODEL_NAME = "Dizex/InstaFoodRoBERTa-NER"        
LABEL_WHITELIST: Set[str] = {"Food", "Dish"}    # entity groups we consider "dishes"
MIN_SCORE = 0.45                                 # confidence threshold (tune as needed)

# Optional: simple sentence splitter (keeps deps light)
SENT_SPLIT_RE = re.compile(r"(?<=[.!?])\s+")

# ---- Lazy singleton loader (loads model once) ----
_ner_pipe = None

def get_ner():
    global _ner_pipe
    if _ner_pipe is None:
        _ner_pipe = pipeline(
            task="ner",
            model=MODEL_NAME,
            aggregation_strategy="simple",
            device=0
        )
    return _ner_pipe

# ---- Public API: extract dish mentions from texts ----
def extract_dishes_from_texts(
    texts: Iterable[str],
    min_score: float = MIN_SCORE,
    label_whitelist: Set[str] = LABEL_WHITELIST
) -> Dict[str, Dict[str, object]]:
    """
    Returns a dict keyed by dish name:
    {
      "spaghetti carbonara": {
         "count": 3,
         "mentions": ["...sentence...", "..."],
         "avg_conf": 0.73
      },
      ...
    }
    """
    ner = get_ner()
    dish_map: Dict[str, Dict[str, object]] = {}

    for full_text in texts:
        # Split into sentences (helps precision and makes snippets nice)
        sentences = SENT_SPLIT_RE.split(full_text.strip()) if full_text else []
        for sent in sentences:
            if not sent:
                continue
            ents = ner(sent)
            for e in ents:
                group = e.get("entity_group") or e.get("entity")  # HF varies by model
                score = float(e.get("score", 0.0))
                word  = (e.get("word") or "").strip()

                if not word or group not in label_whitelist or score < min_score:
                    continue

                dish = normalize_dish(word)
                entry = dish_map.setdefault(dish, {"count": 0, "mentions": [], "avg_conf_sum": 0.0})
                entry["count"] += 1
                # Keep a few representative snippets (cap to avoid bloat)
                if len(entry["mentions"]) < 10:
                    entry["mentions"].append(sent.strip())
                entry["avg_conf_sum"] += score

    # Finalize averages
    for dish, entry in dish_map.items():
        cnt = max(entry["count"], 1)
        entry["avg_conf"] = round(entry.pop("avg_conf_sum") / cnt, 3)

    return dish_map

# ---- Light normalization for dish strings ----
def normalize_dish(s: str) -> str:
    s = s.lower()
    s = re.sub(r"\s+", " ", s).strip()
    # optional: remove trailing punctuation
    s = re.sub(r"[^\w\s\-’']", "", s)
    return s

In [ ]:
from transformers import pipeline

ner = pipeline("ner", model="Dizex/FoodBaseBERT-NER", aggregation_strategy="simple")
# NER = Named Entity Recognition

# function to convert entities to list of strings
def conv_to_food_string(text, entities: list[dict]) -> list[str]:
        """
        Return the extracted food entities as strings. 
        Example: conv_to_food_string(example, ner_entity_results)
        """
        ents = []
        for ent in entities:
            e = {"start": ent["start"], "end": ent["end"], "label": ent["entity_group"]}
            if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
                ents[-1]["end"] = e["end"]
                continue
            ents.append(e)
        return [text[e["start"]:e["end"]] for e in ents]

reviews = [
    "The spaghetti carbonara was amazing!",
    "The tiramisu was creamy and delicious.",
    "I didn’t like the lasagna; too salty.",
    "Their Margherita pizza is the best in town!",
    "Service sehr herzlich und locker, Essen sehr frisch und schmackhaft. Die Fisch Buns sowie das Spicy Shrimp Popcorn sind wirklich toll. Wir kommen wieder, wenn wir in Berlin sind."
    ]

for r in reviews:
    entities = ner(r)
    dishes = conv_to_food_string(r, entities)
    print(f"📝 {r}")
    print(f"🍽️ {dishes}\n")

c:\Users\robin\Desktop\Bootcamp_Files\capstone-dishtip-be\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


📝 The spaghetti carbonara was amazing!
🍽️ ['spaghetti carbonara']

📝 The tiramisu was creamy and delicious.
🍽️ ['tiramisu']

📝 I didn’t like the lasagna; too salty.
🍽️ ['lasagna', 'salty']

📝 Their Margherita pizza is the best in town!
🍽️ ['Margherita pizza']

📝 Service sehr herzlich und locker, Essen sehr frisch und schmackhaft. Die Fisch Buns sowie das Spicy Shrimp Popcorn sind wirklich toll. Wir kommen wieder, wenn wir in Berlin sind.
🍽️ ['Buns', 'Spicy Shrimp Popcorn']



In [73]:
sentiment = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

text = "The lasagna was too salty, but i really liked the tiramisu."
print(sentiment(text))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'positive', 'score': 0.8553879261016846}]


In [69]:
from transformers import pipeline
ner = pipeline("ner", model="Dizex/FoodBaseBERT-NER", aggregation_strategy="simple")

reviews = "Service sehr herzlich und locker, Essen sehr frisch und schmackhaft. Die Fisch Buns sowie das Spicy Shrimp Popcorn sind wirklich toll. Wir kommen wieder, wenn wir in Berlin sind."


food1 = ner(reviews, aggregation_strategy="simple")
print(food1)


c:\Users\robin\Desktop\Bootcamp_Files\capstone-dishtip-be\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'entity_group': 'FOOD', 'score': 0.39927837, 'word': 'Buns', 'start': 79, 'end': 83}, {'entity_group': 'FOOD', 'score': 0.8851886, 'word': 'S', 'start': 94, 'end': 95}, {'entity_group': 'FOOD', 'score': 0.87498945, 'word': '##pic', 'start': 95, 'end': 98}, {'entity_group': 'FOOD', 'score': 0.8441721, 'word': '##y Shrimp Popcorn', 'start': 98, 'end': 114}]


# Review normalization 

In [6]:
REVIEW_SCHEMA = {
    "source": None,        # 'google', 'blog', 'yelp', etc.
    "restaurant": None,    # restaurant name
    "author": None,        # reviewer/blogger name
    "text": None,          # review text content
    "rating": None,        # numeric rating (1–5) or None
    "date": None,          # ISO date string
    "url": None,           # source URL if available
    "language": None,      # optional: helps later with translation
    "id": None,            # optional: unique id (e.g. hash of text+source)
}

FIELD_MAPS = {
    "google": {
        "author_name": "author",
        "text": "text",
        "rating": "rating",
        "time": "date",
        "url": "url",
        "language": "language",
        "author_url": "url",

    },
    "blog": {
        "provider": "author",
        "description": "text",
        "datePublished": "date",
        "url": "url",
    },
}

# TIP_SCHEMA = {
#    "dish name": None,
#    "review_text": "from_schema",
#    "excerpt": None,
#    "author": "from_schema",
#    "source": "google reviews",
#    "review_link": "from_schema",
#    "pos_score": 0.9,
#    "ranking": 4
# }

In [10]:
def normalise_review(raw: dict, source: str) -> dict:
    """
    Normalize a single review dictionary to match the standard REVIEW_SCHEMA.
    Unrecognized keys are ignored. Missing keys get default values.

    Args:
        raw (dict): Raw review data from source.
        source (str): Must be "google", "blog".
    """
    
    # Start with defaults
    review = REVIEW_SCHEMA.copy()

    # Get the appropriate field mapping for this source
    mapping = FIELD_MAPS.get(source, {})

    # Always record where it came from
    review["source"] = source
    

    # Translate raw keys → schema keys
    for raw_key, value in raw.items():
        # Find which schema field this raw key maps to
        schema_key = mapping.get(raw_key)
        if not mapping:
            print(f"[WARN] No field mapping found for source '{source}'")

        if schema_key and schema_key in review:
            review[schema_key] = value

    return review

In [11]:
all_reviews = []

for r in rest1:
    all_reviews.append(normalise_review(r, "google")) 

all_reviews

[{'source': 'google',
  'restaurant': None,
  'author': 'S. K.',
  'text': 'Don’t get distracted by the name :)\nIt’s a very cool, casual, but also a touch of a fancy place!\nThe display of fish looks really nice, and fresh! The price setting feels a little bit higher than excpected, but overall I would recommend this place and be a regular here.',
  'rating': 5,
  'date': 1757272895,
  'url': 'https://www.google.com/maps/contrib/104531257652878694334/reviews',
  'language': 'en',
  'id': None},
 {'source': 'google',
  'restaurant': None,
  'author': 'Dasha Work',
  'text': 'It was an amazing experience! The service in this restaurant is the best from what I saw in Germany so far.\n\nThey proposed me to have an extra meal for free because I didn’t enjoy a soup (not because it was bad, but because it wasn’t for my taste). I said that I’ll take 2 oysters instead, and the waiter said he will bring 3 :) when he came back, he brought 4 oysters for me for free… I was amazed.\n\nAlso the wait